In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.express as px
from utils.metrics import construct_dataframe_faces, construct_dataframe
from utils.imgs import get_bbox_size
import plotly.graph_objects as go

In [3]:
subsets = [
    'base_one_person_dreamshaper',
    'base_two_people_dreamshaper',
    'ip_one_person',
    'ip_one_person_wp5',
    'ip_one_person_wp25',
    'two_people_faceid_dreamshaper',
]
subset_name = subsets[3]

In [4]:
dfs = pd.concat([construct_dataframe(subsets[0]), construct_dataframe(subsets[3])])

In [5]:
dfs = dfs.groupby('subset')['clip_score'].agg([
('mean', 'mean'),
('std', 'std'),
('stderr', lambda x: x.std() / np.sqrt(x.count()))
])

In [6]:
dfs.head()

,mean,std,stderr
subset,,,
base_one_person_dreamshaper,36.827753,2.745844,0.122798
ip_one_person_wp5,35.095483,3.128406,0.139907


In [7]:
df = construct_dataframe_faces(subsets[0])

In [8]:
df.head()

,subset,image_id,bbox,det_score,face_quality,identity,similarity_cosine,pitch,yaw,roll,clip_score,bbox_size
0,base_one_person_dreamshaper,0,"[478, 189, 636, 383]",0.892645,None,5-nat-man,None,-9.694205,13.569555,-6.482787,40.556099,30652
1,base_one_person_dreamshaper,1,"[466, 138, 599, 313]",0.809491,None,18-nat-man,None,-15.001656,-27.590136,10.093520,38.653530,23275
2,base_one_person_dreamshaper,1,"[159, 397, 182, 431]",0.761878,None,None,None,-6.004250,3.897507,-26.663540,38.653530,782
3,base_one_person_dreamshaper,2,"[412, 202, 536, 371]",0.814545,None,5-bla-wom,None,-5.655305,-7.139374,-0.045704,42.513107,20956
4,base_one_person_dreamshaper,2,"[147, 333, 159, 349]",0.539451,None,None,None,27.467674,-49.507732,-22.514410,42.513107,192


In [6]:
df_raw = construct_dataframe_faces(subset_name)

In [7]:
df = df_raw.drop(columns=['kps'])

In [8]:
df['bbox_size'] = df['bbox'].apply(lambda x: get_bbox_size(x))

In [9]:
df.head()

,subset,image_id,bbox,det_score,face_quality,bbox_size
0,ip_one_person_wp5,0,"[341, 121, 682, 575]",0.904596,0.803629,154814
1,ip_one_person_wp5,1,"[330, 141, 648, 574]",0.906030,0.716358,137694
2,ip_one_person_wp5,2,"[320, 179, 659, 634]",0.885794,0.829250,154245
3,ip_one_person_wp5,3,"[357, 238, 657, 606]",0.869382,0.769410,110400
4,ip_one_person_wp5,4,"[428, 140, 658, 461]",0.825942,0.847799,73830


In [10]:
df = df.sort_values('bbox_size', ascending=False).groupby('image_id').first().reset_index()
y_mean = df['face_quality'].mean()
y_std = df['face_quality'].std()
n = len(df)
# Calculate the standard error
y_se = y_std / np.sqrt(n)

In [13]:
fig = px.histogram(df, x='face_quality', title='Face quality when using just a prompt on single person generation')
fig.update_layout(
        width=1000,  # Set a custom width in pixels
    height=600,
    autosize=True, 
    plot_bgcolor='white',
    paper_bgcolor='white', 
    font=dict(color='black'),  # Set font color to black for better contrast
)
# Add mean line
fig.add_vline(x=y_mean, line_dash="dash", line_color="red", annotation_text=f"Mean: {y_mean:.2f}", 
              annotation_position="top right")

# Add +1 std line
fig.add_vline(x=y_mean - y_se, line_dash="dot", line_color="green", annotation_text=f"", 
              annotation_position="top right")

fig.add_vline(x=y_mean + y_se, line_dash="dot", line_color="green", annotation_text=f"", 
              annotation_position="top right")


In [20]:
250 * 250

62500